# 10. 앙상블 부스팅(Boosting)

## 10.1 핵심 개념

**부스팅은 여러개의 약 학습기(weak leaner)를 순차적으로 학습시켜 잘못 예측한 데이터에 가중치를 증가시켜 점진적으로 개선해 나가며 학습하는 앙상블 기법**입니다. 앞서 살펴본 <u>배깅의 경우 여러 데이터셋에서 학습한 결과를 종합하는 병렬식이였다면, 부스팅은 아서 학습한 결과를 두번째 학습에서 수정하고, 세번째 모델에서 또 수정하는등은 순차적 직렬방식의 앙상블</u>이라고 할 수 있습니다.

![배깅과부스팅](./extrafiles/boosting.png)

## 10.2 scikit-learn

부스팅은 sklearn.ensemble 패키지에 속해 있습니다. 부스팅 메소드로는 AdaBoosting, GradientBoosting 을 지원하고 있습니다.


|sklearn.ensemble|Ensemble Methods|
|:--|:--|
|**skleanr.ensemble.AdaBoostClassifier()** |An AdaBoost classifier. |
|**skleanr.ensemble.AdaBoostRegressor()** |An AdaBoost regressor. |
|skleanr.ensemble.BaggingClassifier() |A Bagging classifier. |
|skleanr.ensemble.BaggingRegressor() |A Bagging regressor. |
|skleanr.ensemble.ExtraTreeClassifier() |An extra-tree classifier. |
|skleanr.ensemble.ExtraTreeRegressor() |An extra-tree regressor. |
|**skleanr.ensemble.GradientBoostingClassifier()** |Gradient Boosting for classification. |
|**skleanr.ensemble.GradientBoostingRegressor()** |Gradient Boosting for regressor.|
|skleanr.ensemble.IsolationForest() |Isolation Forest Algorithm. |

### 가. AdaBoosting

AdaBoosting 에서 **분류 알고리즘으로는 AdaBoostingClassifier 와 회귀분석 알고리즘으로 AdaBoostingRegressor** 가 있습니다. 두 방식의 기본 옵션은 동일하여 **base_estimator 와 n_estiator 가 하이퍼파라미터** 입니다.


### 나. GradientBoosting

GradientBoosting 에서 분류 알고리즘으로는 GradientBoostingClassifier 와 회귀분석 알고리즘으로 GradientBoostingRegressor 가 있습니다. 두 방식의 기본 옵션은 동일하며, Gradient 알고리즘 방식은 학습율(learning_rate)를 설정하여 오차(loss)를 줄이는 방향으로 알고리즘이 진행되기에 옵션 설정이 다고 복잡합니다.

## 10.3 분석 코드

### Part1. 분류(Classification)

In [1]:
# 경고레벨조정
import warnings
warnings.filterwarnings("ignore")

# 데이터 로드
import pandas as pd
data = pd.read_csv("./extrafiles/breast-cancer-wisconsin.csv", encoding='utf-8')

# 컬럼정보 확인
print(data.columns)

# 독립변수/ 종속변수 분리
X = data[['Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses']]
y = data[['Class']]

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['code', 'Clump_Thickness', 'Cell_Size', 'Cell_Shape',
       'Marginal_Adhesion', 'Single_Epithelial_Cell_Size', 'Bare_Nuclei',
       'Bland_Chromatin', 'Normal_Nucleoli', 'Mitoses', 'Class'],
      dtype='object')
Class    0.349609
dtype: float64
Class    0.350877
dtype: float64


**[AdaBoosting 앙상블 모델 적용]**

In [3]:
# AdaBoosting 앙상블 모델 적용
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier(n_estimators = 100, random_state = 0)
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

1.0

In [4]:
# 훈련데이터의 혼동행렬 작성
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]


In [7]:
# 훈련데이터의 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print("분류예측 레포트 : \n", cfreport_train)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       333
           1       1.00      1.00      1.00       179

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [5]:
# 테스트데이터 예측결과 생성
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9532163742690059

In [6]:
# 테스트데이터의 혼동행렬 작성
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[106   5]
 [  3  57]]


In [8]:
# 테스트데이터의 분류 레포트 작성
from sklearn.metrics import classification_report
cfreport_test = classification_report(y_test, pred_test)
print("분류예측 레포트 : \n", cfreport_test)

분류예측 레포트 : 
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.92      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



**[GradientBoosting 앙상블 모델 적용]**

In [10]:
# GradientBoosting 앙상블 모델 적용
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=100, learning_rate = 1.0, max_depth=1, random_state=0)
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

1.0

In [11]:
# 훈련데이터의 혼동행렬 작성
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print("훈련데이터 오차행렬 : \n", confusion_train)

훈련데이터 오차행렬 : 
 [[333   0]
 [  0 179]]


In [12]:
# 테스트데이터 예측결과 생성
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9649122807017544

In [13]:
# 테스트데이터의 혼동행렬 작성
confusion_test = confusion_matrix(y_test, pred_test)
print("테스트데이터 오차행렬 : \n", confusion_test)

테스트데이터 오차행렬 : 
 [[106   5]
 [  1  59]]


**[종합정리]**

부스팅 방법은 오류를 찾아 해결하는 방식으로 해를 찾으르모 훈련데이터에 과적합되는 경향을 보입니다. AdaBoosting, GradientBoosting 모두 과적합 문제가 나타나지만 테스트 데이터에서는 일반적인 수준의 정확도를 보이고 있습니다. SCV 알고리즘외 다른 알고리즘을 사용한다면 보다 좋은 결과도 기대할 수 있습니다.

### Part2. 회귀(Regression)

In [14]:
# 데이터 로드
data2 = pd.read_csv('./extrafiles/house_price.csv', encoding='utf-8')

print(data2.columns)

X = data2[data2.columns[1:5]]
y = data2[['house_value']]

print(X.columns)

# train-test data 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# stratify 효과 - 범주형 변수를 유사한 비율로 train / test 데이터로 분리시켜 준다.
print(y_train.mean())
print(y_test.mean())

# 표준화 작업 - MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)

X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

Index(['housing_age', 'income', 'bedrooms', 'households', 'rooms',
       'house_value'],
      dtype='object')
Index(['income', 'bedrooms', 'households', 'rooms'], dtype='object')
house_value    189260.967812
dtype: float64
house_value    188391.001357
dtype: float64


**[AdaBoosting 앙상블 모델 적용]**

In [15]:
# AdaBoosting 앙상블 모델 적용
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor(random_state=0, n_estimators=100)
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.4353130085971758

In [16]:
# 테스트 데이터 모델 적용
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.43568387094087124

In [17]:
# 회귀분석의 지표 R Square n RMSE
# RMSE (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
import numpy as np
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈  련데이터 RMSE:", np.sqrt(MSE_train))
print("테스트데이터 RMSE:", np.sqrt(MSE_test))

훈  련데이터 RMSE: 71722.42012035428
테스트데이터 RMSE: 71816.41231019037


**[GradientBoosting 앙상블 모델 적용]**

In [19]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=0)
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.6178724780500952

In [20]:
# 테스트 데이터 모델 적용
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5974112241813845

In [21]:
# 회귀분석의 지표 R Square n RMSE
# RMSE (Root Mean Squared Error)
from sklearn.metrics import mean_squared_error
import numpy as np
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print("훈  련데이터 RMSE:", np.sqrt(MSE_train))
print("테스트데이터 RMSE:", np.sqrt(MSE_test))

훈  련데이터 RMSE: 59000.433545962376
테스트데이터 RMSE: 60658.72886338227


**[종합정리]**

회귀문제에서는 기본 모델을 설정한 경우 GradientBoosting 이 AdaBoosting 보다 더 좋은 성능을 보여주었다. 하지만 일반적인 경우 하이퍼파라미터를 조정 하는 경우 AdaBoosting 이 보다 더 좋은 결과를 낸다.